In [1]:
import pandas as pd
import os
from datetime import datetime
import logging
from typing import Dict
from typing import Dict

In [121]:
# Diccionario para los tipos de la data
dict_ColumnsTypes_data = {"Referencia":"str",
                          "Asignación":"str",
                          "Soc.":"str",
                          "Acreedor":"str",
                          "Clave ref.1":"str",
                          "Clave ref.2":"str",
                          "Cuenta":"str",
                          "Clave referencia 3":"str",
                          "Período":"str",
                          "Período":"str",
                          "Cl.coste":"str",
                          "CeBe":"str",
                          "CeBe int.":"str",
                          "Div.":"str",
                          "NIT":"str",
                          "Deudor":"str",
                          "Lib.mayor":"str",
                          "Cta.mayor":"str",
                          "VP":"str",
                          "Fec.Expedi":"str",
                          "Fecha doc.":"str",
                          "Fe.contab.":"str",
                          "ML2":"str",
                          "Importe":"str",
                          "ImpteML2":"str",
                          "Importe en ML3":"str",
                          "Tp.camb.ef.":"str",
                          "MonPa":"str",
                          "Compens.":"str",
                          "Fe.valor":"str",
                          "Fecha pago":"str",
                          "Registrado":"str"}

dict_ColumnsTypes_cabecera = {"Referencia":"str",
                              "Texto cab.documento":"str",
                              "Anul.con":"str",
                              "Tipo cambio":"str"}

In [199]:
archivo_input = "01.Input\Detalle.xlsx"
archivo_cabecera = "01.Input\Cabecera.xlsx"
df_importado = pd.read_excel(archivo_input, dtype=dict_ColumnsTypes_data)
df_cabecera = pd.read_excel(archivo_cabecera, dtype=dict_ColumnsTypes_cabecera)

In [200]:
# Cruce de df_importado con df_cabecera
df_importado = df_importado.merge(df_cabecera, on='Referencia', how='left')

In [201]:
# Transformaciones previas a la validación
df_importado['Referencia_temp'] = df_importado['Referencia'].apply(lambda x: x[:-1] if x.endswith('I') else x)
df_importado['Tipo cambio'] = df_importado['Tipo cambio'].apply(lambda x: round(float(f"{x.replace('.', '').replace(',', '')[:-5]}.{x.replace('.', '').replace(',', '')[-5:]}"), 2))

In [202]:
# Validar referencias que tienen producción e IVA calculado con diferente TRM
trm_unicas = df_importado.groupby('Referencia_temp').agg(TRM_Unicas=('Tipo cambio', 'nunique')).reset_index()

# Filtrar solo registros de IVA para asignar la tasa correcta
trm_iva = df_importado.loc[df_importado['Referencia'].str.endswith('I')].drop_duplicates(subset=['Referencia'])[['Referencia_temp','Tipo cambio']]
trm_iva = trm_iva.rename(columns={'Tipo cambio': 'Tasa_Correcta'})

In [203]:
# Cruce de df_importado con trm_unicas para llevar la columna TRM_Unicas a la BD principal
df_importado = df_importado.merge(trm_unicas, on='Referencia_temp', how='left')

# Cruce de df_importado con trm_iva para asignar la tasa correcta
df_importado = df_importado.merge(trm_iva, on='Referencia_temp', how='left')

# Corregir columna Tasa_Correcta cuando la referencia no tenga IVA
df_importado['Tasa_Correcta'] = df_importado.apply(lambda x: x['Tipo cambio'] if pd.isna(x['Tasa_Correcta']) else x['Tasa_Correcta'], axis=1)

In [204]:
# Filtrar aquellas referencias que tienen TRM diferentes
df_depurado = df_importado.loc[df_importado['TRM_Unicas'] != 1]
df_depurado = df_importado
df_depurado = df_depurado.sort_values(by=['Referencia', 'Lib.mayor'])

In [205]:
df_depurado.to_excel('Prueba.xlsx', index=False)